In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
# Remove index_col if there's no extra column or index column at the starting
df_common=pd.read_csv("/content/drive/MyDrive/gene_ext/del/30/common_wd30.csv", index_col="Unnamed: 0")
df_protein_file = pd.read_csv("/content/drive/MyDrive/gene_ext/ind_protein/ind_chr30_prot.csv")

In [ ]:
df_common.shape

(1913, 12)

In [ ]:
# df_common.columns

In [ ]:
# Add column as Length of variations (LOV) in common_wd1.csv
df_common["LOV"]=abs(df_common["endpos.r"]-df_common["startpos.r"])

In [ ]:
df_common.shape

(1913, 13)

In [ ]:
df_common

,#CHROM,POS,REF,ALT,chr.r,startpos.r,endpos.r,seq.r,seq.q,startpos.q,endpos.q,chr.q,LOV
0,NC_037357.1,5171760,CT,C,NC_037357.1,5171760,5171761,CT,C,598897,598897,NC_032679.1,1
1,NC_037357.1,5171774,GT,G,NC_037357.1,5171774,5171775,GT,G,598910,598910,NC_032679.1,1
2,NC_037357.1,5216510,AG,A,NC_037357.1,5216510,5216511,AG,A,643467,643467,NC_032679.1,1
3,NC_037357.1,6129364,GT,G,NC_037357.1,6129364,6129365,GT,G,1049068,1049068,NC_032679.1,1
4,NC_037357.1,6210296,TA,T,NC_037357.1,6210296,6210297,TA,T,1129905,1129905,NC_032679.1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920,NC_037357.1,135060534,AG,A,NC_037357.1,135060534,135060535,AG,A,86327114,86327114,NC_032679.1,1
1921,NC_037357.1,135060914,AG,A,NC_037357.1,135060914,135060915,AG,A,86327492,86327492,NC_032679.1,1
1922,NC_037357.1,135763649,AT,A,NC_037357.1,135763649,135763650,AT,A,87030217,87030217,NC_032679.1,1
1923,NC_037357.1,135763689,GC,G,NC_037357.1,135763689,135763690,GC,G,87030257,87030257,NC_032679.1,1


In [ ]:
df_common.to_csv("common30_LOV.csv")

In [ ]:
df_protein_file.shape

(993, 11)

In [ ]:
def map_keywords(str_sentence):
    list_keywords = [
        "immunoglobulin","immunoreceptor","autoimmune","TLR","IgG",
        "autoimmune","autophagy","immunogen","immune","innate","T-cell","NF-kappa", "antigen",
        "B-cell","lymphocyte","histocompatibility","CD24","CD4","LY96", "BCR",
        "IFIT3","PGLYRP1","NKG2D","UL16","leukocyte","cytokine", "interleukin","interferon"
        "antimicrobial peptide","beta-defensin 2","IL16","IL2","chemokine", "antibody"
    ]
    bool_found = bool(
        [
            i for i in list_keywords if i.lower() in str_sentence.lower()
        ]
    )
    return bool_found

In [ ]:
df_protein_file["keyword_found"] = df_protein_file.apply(lambda row: map_keywords(row["Protein Name"]),axis=1)

In [ ]:
df_protein_file[df_protein_file["keyword_found"]==True].shape

(61, 12)

In [ ]:
df_protein_file[df_protein_file["keyword_found"]==True].drop_duplicates().shape

(61, 12)

In [ ]:
# df_protein_file_imm=df_protein_file[df_protein_file["keyword_found"]==True].drop_duplicates()

In [ ]:
df_protein_file["start_of_next_gene"] = df_protein_file["Start"].shift(-1).fillna(0)

In [ ]:
# df_protein_file.head()

In [ ]:
df_protein_file.to_csv("protein30.csv")

In [ ]:
a=0
b=0
c=0
d=0
e=0
f=0

In [ ]:
from tqdm import tqdm

In [ ]:
final_df = pd.DataFrame(columns=list(df_protein_file.columns)+["gene_type"]+["startpos.q"]+["endpos.q"]+["seq.r"]+["seq.q"]+["LOV"])
for i, row in tqdm(df_common.iterrows()):
    # Check for the at start
    if df_protein_file[(df_protein_file["Start"]==row["startpos.q"]) & (df_protein_file["Stop"]>row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Start"]==row["startpos.q"]) & (df_protein_file["Stop"]>row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "at_start"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        a=a+1
    # check for the at end
    elif df_protein_file[(df_protein_file["Start"]<row["startpos.q"]) & (df_protein_file["Stop"]==row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Start"]<row["startpos.q"]) & (df_protein_file["Stop"]==row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "at_end"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        b=b+1
    # Check for the within gene
    elif df_protein_file[(df_protein_file["Start"]<row["startpos.q"]) & (df_protein_file["Stop"]>row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Start"]<row["startpos.q"]) & (df_protein_file["Stop"]>row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "within_gene"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        c=c+1
    # Check for the near start gene with integenic
    elif df_protein_file[(df_protein_file["Start"]==row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Start"]==row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "gene_start_with variation end"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        d=d+1
    # Check for the near stop gene with intergenic
    elif df_protein_file[(df_protein_file["Stop"]==row["startpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Stop"]==row["startpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "gene_stop_ with variation start"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        e=e+1
    # Check for the intergenic gene
    elif df_protein_file[(df_protein_file["Stop"]<row["startpos.q"]) & (df_protein_file["start_of_next_gene"]>row["endpos.q"])].shape[0]>=1:
        tmp_df = df_protein_file[(df_protein_file["Stop"]<row["startpos.q"]) & (df_protein_file["start_of_next_gene"]>row["endpos.q"])].copy(deep=True)
        tmp_df["gene_type"] = "intergenic_gene"
        tmp_df["startpos.q"] = row["startpos.q"]
        tmp_df["endpos.q"] = row["endpos.q"]
        tmp_df["seq.r"] = row["seq.r"]
        tmp_df["seq.q"] = row["seq.q"]
        tmp_df["LOV"] = row["LOV"]
        final_df = pd.concat([final_df,tmp_df]).reset_index(drop=True)
        f=f+1
    else:
        pass


1913it [00:19, 99.72it/s]


In [ ]:
print(a)
print(b)
print(c)
print(d)
print(e)
print(f)

0
0
381
0
0
1532


In [ ]:
final_df.shape

(2439, 19)

In [ ]:
final_df.drop_duplicates().shape

(2439, 19)

In [ ]:
final_df.to_csv("genesv30.csv")

In [ ]:
# non coding region
final_df1 = pd.DataFrame(columns=df_common.columns)
for index, row in tqdm(df_common.iterrows()):
    # Check if 'startpos.q' is present in any row of file2
    if row['startpos.q'] not in final_df['startpos.q'].values:
        # Append the row to the DataFrame if not found
        final_df1 = final_df1.append(row, ignore_index=True)

# Save the DataFrame with rows not found in file2 to a new CSV file
final_df1.to_csv("non_coding30.csv", index=False)

1913it [00:00, 4834.17it/s]


In [ ]:
final_df1.shape

(0, 13)

In [ ]:
final_df1.drop_duplicates().shape

(0, 13)

In [ ]:
imm = pd.DataFrame
imm=final_df[final_df["keyword_found"]==True]

In [ ]:
imm.shape

(286, 19)

In [ ]:
imm.drop_duplicates().shape

(286, 19)

In [ ]:
imm.drop_duplicates().to_csv("immv30.csv")

In [ ]:
# counting the values
# Create a new DataFrame with unique combinations of 'Locus', 'startpos.q', and 'endpos.q'
locus_counts = final_df.drop_duplicates(subset=['Locus', 'startpos.q', 'endpos.q'])

In [ ]:
locus_counts.shape

(1913, 19)

In [ ]:
locus_counts.to_csv("locus_counts30.csv")

In [ ]:
locus_counts_imm = imm.drop_duplicates(subset=['Locus', 'startpos.q', 'endpos.q'])

In [ ]:
locus_counts_imm.shape

(286, 19)

In [ ]:
locus_counts_imm.to_csv("locus_counts_imm30.csv")

In [ ]:
gene_count=locus_counts.groupby(["gene_type"]).size().reset_index(name="count")

In [ ]:
gene_count

,gene_type,count
0,intergenic_gene,1532
1,within_gene,381


In [ ]:
gene_count.to_csv("gene_count30.csv")

In [ ]:
loc_imm_count=locus_counts_imm.groupby(by=["Locus","gene_type"]).size().reset_index(name="count")
# loc_imm_count.columns=["Locus", "gene_type", "count"]

loc_imm_count.to_csv("immlocus_gene_type_counts30.csv", index=False)

In [ ]:
loc_imm_count

,Locus,gene_type,count
0,CDR1,intergenic_gene,73
1,LOC109554781,intergenic_gene,194
2,LOC109554808,intergenic_gene,2
3,LOC109554932,intergenic_gene,1
4,LOC109554955,intergenic_gene,2
5,LOC109555064,intergenic_gene,4
6,LOC109555202,intergenic_gene,5
7,MAGEB16,intergenic_gene,3
8,MCTS1,intergenic_gene,2


In [ ]:
# Calculate value counts
count = locus_counts["gene_type"].value_counts().reset_index()
count.columns = ["gene_type", "Count"]
count.loc[len(count.index)]=["immune_gene", len(locus_counts_imm)]
count.loc[len(count.index)]=["non_coding_gene", len(final_df1)]


# Save the result to a CSV file
count.to_csv("delcounts30.csv", index=False)

In [ ]:
import shutil

shutil.copy("protein30.csv","/content/drive/MyDrive/gene_ext/del/30/new")
shutil.copy("common30_LOV.csv","/content/drive/MyDrive/gene_ext/del/30/new/")
shutil.copy("genesv30.csv","/content/drive/MyDrive/gene_ext/del/30/new/")
shutil.copy("immv30.csv","/content/drive/MyDrive/gene_ext/del/30/new/")
shutil.copy("locus_counts30.csv","/content/drive/MyDrive/gene_ext/del/30/new/")
shutil.copy("locus_counts_imm30.csv","/content/drive/MyDrive/gene_ext/del/30/new/")
shutil.copy("immlocus_gene_type_counts30.csv","/content/drive/MyDrive/gene_ext/del/30/new/")
shutil.copy("gene_count30.csv","/content/drive/MyDrive/gene_ext/del/30/new/")
shutil.copy("delcounts30.csv","/content/drive/MyDrive/gene_ext/del/30/new/")

'/content/drive/MyDrive/gene_ext/del/30/new/delcounts30.csv'

In [ ]:
# immune genes information - deletion
import shutil

Chr_list=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, "X"]
# Chr_list = [1]
for chr in Chr_list:
  print(f"chromosome:{chr}")
  in_file1=f"/content/drive/MyDrive/COW_GENOME_RESEARCH/Objective_1/Whole_Genome_Analysis_Existing_Data/gene_ext/del/{chr}/new/immloc_count_within_gene{chr}.csv"
  outloc=f"/content/drive/MyDrive/COW_GENOME_RESEARCH/Objective_1/Whole_Genome_Analysis_Existing_Data/gene_ext/del/{chr}/new"
  imf1=pd.read_csv(in_file1)

  # extract the immune has how many variations and its gene type
  locwithin_imm_count=imf1.groupby(by=["Locus"]).size().reset_index(name="count")

  # Group by 'Locus' and calculate the sum of 'LOV'
  grouped = imf1.groupby('Locus')['LOV'].sum().reset_index()

  # Merge the two files on the 'Locus' column
  merged_df = pd.merge(locwithin_imm_count, grouped, on='Locus')

  # Rename columns for clarity
  merged_df.rename(columns={'count': 'No. of Variations', 'LOV': 'Length of Variations'}, inplace=True)

  chr_col=f"{chr}"

  merged_df["Chromosome"] = chr_col

  # Reorder columns
  reorder_df = merged_df[['Chromosome','Locus', 'No. of Variations', 'Length of Variations']]

  out_f1=f"imm_var_count{chr}.csv"
  reorder_df.to_csv(out_f1)

  shutil.copy(out_f1, outloc)


chromosome:1
chromosome:2
chromosome:3
chromosome:4
chromosome:5
chromosome:6
chromosome:7
chromosome:8
chromosome:9
chromosome:10
chromosome:11
chromosome:12
chromosome:13
chromosome:14
chromosome:15
chromosome:16
chromosome:17
chromosome:18
chromosome:19
chromosome:20
chromosome:21
chromosome:22
chromosome:23
chromosome:24
chromosome:25
chromosome:26
chromosome:27
chromosome:28
chromosome:29
chromosome:X
